## Using NexMotion API to Control MiniBot
### Pre-requirement:
* Python2.7 32bit version (NexMotion.dll is 32bit version)
* Create MiniBot Configuration from NexMotion Studio (NexMotionNcf.ncf)
* For controlling real MiniBOT, do homing process in NexMotion Studio first.

#### 1. Include Necessary Libarary with environment variable

In [1]:
%config IPCompleter.greedy=True
# include nexmotion library
import nexmotion as nm
from nexmotion.errors import *
from nexmotion.constants import *
import numpy as np
import pandas as pd
dll_path = None
ini_path = None

#### 2. Start Device

In [2]:
if dll_path is None:
    nmc = nm.Control()  # will use default path for dll and ini file
else:
    nmc = nm.Control(dll_path=dll_path)
    nmc.setIniPath(ini_path)
ret = nmc.deviceOpenup(DEVICE_TYPE_SIMULATOR)  # use DEVICE_TYPE_ETHERCAT to connect real Minibot
if ret != nm.errors.SUCCESS:
    print( "NMC_DeviceOpenUp error, error code =", ret)

Dynamics library version = 10121027 ( 1 , 1 , 2 , 1027 )
device id = 0


#### 3. Check Device State
Get the Device State.
Here are the 4 possible state. If nothing wrong, the device state will be **4**

| Name   |      Value      |  Description |
|--------|:-------------:|------:|
| NMC_DEVICE_STATE_INIT      | 1 | Initial |
| NMC_DEVICE_STATE_READY     | 2 | Ready |
| NMC_DEVICE_STATE_ERROR     | 3 | Error |
| NMC_DEVICE_STATE_OPERATION | 4 | Operation |

In [3]:
ret = nmc.deviceGetState()
if ret != SUCCESS:
    print( "NMC_DeviceGetState error, error code =", ret)
else:
    if nmc.devState_.value != DEVICE_STATE_OPERATION:
        print( "device state is not OPERATION" )
    else:
        print( "device state =", nmc.devState_.value)


device state = 4


#### 4. Display Virtual Robot for visualization

In [4]:
nmc.group3DShow(top_=True)
nmc.group3DDrawPath(enable=True)

0

#### 5. Get device information: Count of group

In [5]:
nmc.deviceGetGroupCount()
print( "count of group index", nmc.numGroup_.value)

count of group index 1


#### 6. Get device information: Count of Group Axis
For MiniBot, the below code will output: 6

In [6]:
nmc.deviceGetGroupAxisCount()
print( "group index", nmc.index_.value, ": count of group axis = ", nmc.numGroupAxis_.value)

group index 0 : count of group axis =  6


#### 7. Clean alarm of drives of each group
When sending movement command (such as moveLine, groupLine), the robot may not execute correctly (such as desired position is outside its workspace or in singular point. Then, the controller will set group state to `GROUP_ERROR_STOP`).
These two are useful command to clean alarm or reset state of robot.

In [4]:
nmc.groupResetDriveAlmAll()
nmc.groupResetState()

0

## 8. Check group state
Here are all possible group (movement) state:
Get the state of group. Please refer to the below table for details.

|Value | State | Description | 
|------|:-------:|-------------:| 
| 0 | GROUP_DISABLE     | A group axis is disabled. | 
| 1 | GROUP_STAND_STILL | All group axes are enabled. | 
| 2 | GROUP_STOPPED     | After NMC_GroupStop() is called, the group is stopped. |
| 3 | GROUP_STOPPING    | After NMC_GroupStop() is called, the group is stopping. |
| 4 | GROUP_MOVING      | The group is moving. |
| 5 | GROUP_HOMING      | The group is homing. |
| 6 | GROUP_ERROR_STOP  | An error is occurred in a group axis. |

##### Note: if Group State == 6 (error stop), please run step 7 to reset state.
##### Note: if Group State == 0 (disable), please run step 9 to servo on in order to move robot arm.


In [6]:
ret = nmc.groupGetState()
if ret != SUCCESS:
    print( "groupGetState error, error code =", ret)
else:
    print( "Group State = ", nmc.groupState_.value)

Group State =  1


#### 9. Enable all axes of group (Servo On)
MiniBot will make a sound when it is enabled.
And, group sate will be changed (could run step **8** to check again)  
Note: need to Enable before running any movement command

In [5]:
ret = nmc.groupEnable()
if ret != SUCCESS:
    print( "NMC_DeviceEnableAll error, error code =", ret)
else:
    print( "enable all done" )

enable all done


#### 10. Show Actual Joint Position
This step can be called to check actual joint values after you move or home the Minibot.  
For controlling real MiniBOT, make sure you do homing process (or this command will not show correctly)

In [8]:
jntPos = [0.] * 6
ret = nmc.groupGetActualPosAcs(jntPos)
if ret != SUCCESS:
    print( "NMC_GroupGetActualPosAcs error, error code =", ret)
else:
    print( "NMC_GroupGetActualPosAcs:")
    for i in range(0, 6, 1):
        print( "   Joint", i ,"val:", '%.3f' % jntPos[i])

NMC_GroupGetActualPosAcs:
   Joint 0 val: 0.003
   Joint 1 val: 26.243
   Joint 2 val: 0.003
   Joint 3 val: 0.004
   Joint 4 val: -29.028
   Joint 5 val: 0.009


## 11. Execute Group PTP motion to a pre-define position
##### Note: Please make sure all the joints are finish homing (if controlling real MiniBOT)

In [7]:
desPos = [0, 90, 0, 0, -90, 0]
nmc.groupPtpAcsAll(desPos)

0

#### 12. Show Actual Cartesian Position 
This step can be called to check actual cartesian values.

In [9]:
#get current cartesian value
cartPos = [0.] * 6
ret = nmc.groupGetActualPosPcs(cartPos)
if ret != SUCCESS:
    print( "groupGetActualPosPcs error, error code =", ret)
else:
    print( "groupGetActualPosPcs:")
    coord_list = ['x', 'y', 'z', 'a', 'b', 'c']
    for i in range(0, 6, 1):
        print( "\t", coord_list[i] ,"val:", '%.3f' % cartPos[i])

groupGetActualPosPcs:
	x val: 400.877
	y val: 0.018
	z val: 313.021
	a val: -0.007
	b val: -0.423
	c val: -179.998


## 13. Execute Group Line motion to a pre-define position
##### `Note: Please make sure all the joints are finish homing`

In [10]:
pos = [0.] * 6
nmc.groupGetActualPosPcs(pos)
pos[0] += 50; pos[1] += -100; pos[2] += -150
nmc.groupLine(pos)

0

#### 15. record current pose (Joint space and Cartesian space)

In [13]:
nmc.recordPoint()

0

#### 16. show recorded points

In [12]:
df = pd.DataFrame(data=nmc.pnt_list, 
                  columns=['j1','j2','j3','j4','j5','j6','x','y','z','a','b','c'])
#df.to_csv("temp_pnt.csv")
print( df)

          j1         j2            j3            j4         j5            j6  \
0  30.465682  33.361364 -3.349773e+00  2.168404e-19 -30.012750  3.046275e+01   
1  23.806364  37.009091 -1.116409e+01  2.168404e-19 -25.846875  2.380500e+01   
2   0.000682  90.000000 -1.525466e-13  2.168404e-19 -89.998875 -8.046515e-15   
3 -23.197500  56.781818 -2.507045e+00  2.168404e-19 -54.274500 -2.319750e+01   
4 -23.961136  43.082045 -4.224545e+00  2.168404e-19 -38.857500 -2.396250e+01   

            x           y           z             a             b           c  
0  339.999206  200.000626  200.010617  2.931813e-03  9.990888e-04  179.999412  
1  339.995877  150.001159  200.006932  1.363625e-03  1.715483e-03  179.999243  
2  250.001865    0.002975  564.000000  6.818182e-04 -1.125000e-03  180.000000  
3  350.001493 -149.992755  364.000805  2.344101e-10 -2.506779e-04  179.999893  
4  359.999931 -159.989792  264.001042  1.363636e-03  2.983448e-11 -180.000000  


## 17. move PTP like GRC

In [22]:
nmc.movePTP(1)

0

## 18 moveLine like GRC

In [12]:
nmc.moveLine(4)

0

## 19. Halt Robot Movement

In [41]:
nmc.groupHalt()

0

## 20. Set DO value
Use Digital Output to control gripper or suctions.

In [31]:
DO_list = [[0,2,4],[]]  # Turn digital oupt 0, 2, 4 to `on` and other to `off`
nmc.writeOutputMemory(DO_list)

0

### Write Script 

In [43]:
import time
nmc.movePTP(0)
nmc.movePTP(1)
nmc.movePTP(2)
nmc.moveLine(3)
nmc.writeOutputMemory([[2],[]])
for i in xrange(2):
    nmc.moveLine(4)
    nmc.moveLine(3)
nmc.writeOutputMemory([[],[]])
nmc.moveLine(2)
nmc.moveLine(1)
nmc.movePTP(0)


0

### Save and Read Point from csv file

In [21]:
nmc.savePoint('test_pnt.csv')

In [11]:
nmc.readPoint('test_pnt.csv')

['0', '30.465681818181217', '33.361363636356245', '-3.349772727272639', '2.168404344971009e-19', '-30.01275000000994', '30.46275000000095', '339.999205647997', '200.0006262045144', '200.01061662089927', '0.0029318130568759264', '0.000999088794146131', '179.99941236630718']
['1', '23.80636363636341', '37.00909090908368', '-11.164090909090831', '2.168404344971009e-19', '-25.84687500001049', '23.805000000000696', '339.9958768768381', '150.00115921347776', '200.00693223088035', '0.0013636250331441745', '0.0017154833568307926', '179.99924320285496']
['2', '0.0006818181813115057', '89.99999999999153', '-1.5254659514740698e-13', '2.168404344971009e-19', '-89.99887500001594', '-8.04651484331842e-15', '250.00186530294343', '0.002975015238154895', '564.0000000182752', '0.0006818181813115058', '-0.001124999975428993', '180.0']
['3', '-23.197500000001327', '56.781818181808895', '-2.5070454545455974', '2.168404344971009e-19', '-54.274500000021696', '-23.19750000000073', '350.00149264813945', '-149.

### Compute Base transformation pose

In [14]:
coordTrans = [0.]*6
nmc.baseCalib_2p(nmc.pnt_list[0][6:], nmc.pnt_list[1][6:], coordTrans)
print coordTrans

[339.999205647997, 200.0006262045144, 200.01061662089927, -90.0038145314253, -0.0, 0.0]


### Get Transformation matrix

In [18]:
TrMat = nm.pose2matrix(coordTrans)
print np.around(TrMat, decimals=2)

[[   0.      1.      0.    340.  ]
 [  -1.      0.      0.    200.  ]
 [   0.      0.      1.    200.01]
 [   0.      0.      0.      1.  ]]


In [24]:
# From robot-base position transform to pixel-based position
pos = [0.] * 6
nmc.groupGetActualPosPcs(pos)
print pos
TrInv = np.linalg.inv(TrMat)
tcpMat = nm.pose2matrix(pos) # [340, 200, 200, 0, 0, 180]
print np.around(TrInv.dot(tcpMat), decimals=1)
print np.around(TrInv.dot(np.mat([340, 150, 200, 1]).transpose()), decimals=1)

# From pixel-base position transform to robot-base position
pixMat = nm.pose2matrix([50, 0, 0, 0, 0, 0])
print np.around(TrMat.dot(pixMat), decimals=2)
print np.around(TrMat.dot(np.mat([250, 0, 0, 1]).transpose()), decimals=2)

[339.9926569616489, 149.99973863444632, 200.00251013684613, -0.0008864073264666338, 0.0033685271050913324, 179.99851379333722]
[[  0.   1.   0.  50.]
 [  1.   0.   0.   0.]
 [  0.   0.  -1.   0.]
 [  0.   0.   0.   1.]]
[[ 50.]
 [  0.]
 [  0.]
 [  1.]]
[[   0.      1.      0.    340.  ]
 [  -1.      0.      0.    150.  ]
 [   0.      0.      1.    200.01]
 [   0.      0.      0.      1.  ]]
[[ 339.98]
 [ -50.  ]
 [ 200.01]
 [   1.  ]]


#### 21. Disable all axes of group (Servo Off)
MiniBot will make a sound when it is disabled.

In [25]:
nmc.groupDisable()

0

#### 22. Shutdown device

In [26]:
nmc.deviceShutdown()

0